In [ ]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'diffusers'

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the tokenizer and text model for the prompt
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
text_model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")

# Load the stable diffusion model
pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipeline = pipeline.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

NameError: name 'StableDiffusionPipeline' is not defined

In [ ]:
def generate_image_from_text(prompt):
    # Tokenize and encode the text
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate the image
    with torch.no_grad():
        image = pipeline(inputs["input_ids"]).images[0]

    return image

# Example usage
prompt = "A fantasy landscape with mountains and a river"
image = generate_image_from_text(prompt)

# Display the image
plt.imshow(image)
plt.axis('off')
plt.show()


In [ ]:
pip install Flask


In [ ]:
from flask import Flask, request, render_template
import torch
from transformers import CLIPTokenizer, CLIPTextModel
from diffusers import StableDiffusionPipeline
import io
from PIL import Image

app = Flask(__name__)

# Load models
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")
pipeline = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipeline = pipeline.to(device)

def generate_image(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        image = pipeline(inputs["input_ids"]).images[0]
    return image

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        prompt = request.form['prompt']
        image = generate_image(prompt)
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='PNG')
        img_byte_arr = img_byte_arr.getvalue()
        return render_template('index.html', img_data=img_byte_arr)
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Text to Image Generator</title>
</head>
<body>
    <h1>Text to Image Generator</h1>
    <form method="POST">
        <label for="prompt">Enter Text Description:</label>
        <input type="text" id="prompt" name="prompt" required>
        <button type="submit">Generate Image</button>
    </form>
    {% if img_data %}
        <h2>Generated Image:</h2>
        <img src="data:image/png;base64,{{ img_data|tojson|safe }}" alt="Generated Image">
    {% endif %}
</body>
</html>


In [ ]:
python app.py
